In [57]:
# すべてのascii文字で辞書を作る
import string
all_chars = string.printable
vocab_size = len(all_chars)
vocab_dict = dict((c, i) for (i, c) in enumerate(all_chars))

# 文字列を数値のリストに変換する関数
def str2ints(s, vocab_dict):
    return [vocab_dict[c] for c in s]

# 数値のリストを文字列に変換する関数
def ints2str(x, vocab_array):
    return "".join([vocab_array[i] for i in x])

In [58]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset,
                             DataLoader,
                             TensorDataset)
import tqdm

In [59]:
class ShakespeareDataset(Dataset):
    def __init__(self, path, chunk_size=200):
        # ファイルを読み込み、数値のリストに変換する
        data = str2ints(open(path).read().strip(), vocab_dict)
        # Tensorに変換し、splitする
        data = torch.tensor(data, dtype=torch.int64).split(chunk_size)
        # 最後のshunkにながさをチェックして足りない場合には捨てる
        if len(data[-1]) < chunk_size:
            data = data[:-1]
            
        self.data = data
        self.n_chunks = len(self.data)
    
    def __len__(self):
        return self.n_chunks
    
    def __getitem__(self, idx):
        return self.data[idx]

In [60]:
ds = ShakespeareDataset("ch5data/tinyshakespeare.txt", chunk_size=200)
loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=4)

In [61]:
"""
文章生成モデルの構築
"""
class SequenceGenerationNet(nn.Module):
    def __init__(self, num_embeddings, 
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, 
                            hidden_size,
                            num_layers,
                            batch_first=True,
                            dropout=dropout)
        # Linerのoutputのサイズは最初のEmbeddingの
        # inputサイズと同じnum_embeddings
        self.linear = nn.Linear(hidden_size, num_embeddings)

    def forward(self, x, h0=None):
        x = self.emb(x)
        x, h = self.lstm(x, h0)
        x = self.linear(x)
        return x, h

In [62]:
"""
文章を生成する関数の作成
"""
def generate_seq(net, start_phrase="The King said ",
                 length=200, temperature=0.8, device="cpu"):
    # モデルを評価モードにする
    net.eval()
    # 出力の数値を格納するリスト
    result = []
    
    # 開始文字列をTensorに変換
    start_tensor = torch.tensor(
        str2ints(start_phrase, vocab_dict),
        dtype=torch.int64
    ).to(device)
    # 先頭にbatch次元を付ける
    x0 = start_tensor.unsqueeze(0) 
    # RNNに通して出力と新しい内部状態を得る
    o, h = net(x0)
    # 出力を(正規化されていない)確率に変換
    out_dist = o[:, -1].view(-1).exp()
    # 確率から実際の文字のインデクスをサンプリング
    top_i = torch.multinomial(out_dist, 1)[0]
    # 結果を保存
    result.append(top_i)
    
    # 生成された結果を次々にRNNに入力していく
    for i in range(length):
        inp = torch.tensor([[top_i]], dtype=torch.int64)
        inp = inp.to(device)
        o, h = net(inp, h)
        out_dist = o.view(-1).exp()
        top_i = torch.multinomial(out_dist, 1)[0]
        result.append(top_i)
        
    # 開始文字列と生成された文字列をまとめて返す
    return start_phrase + ints2str(result, all_chars)

In [63]:
"""
文章を生成する関数の作成
"""
from statistics import mean

net = SequenceGenerationNet(vocab_size, 20, 50,
                            num_layers=2, dropout=0.1)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
# 多クラスの識別で問題なのでSoftmaxCrossEntropyLossが損失関数となる
loss_f = nn.CrossEntropyLoss()

for epoch in range(50):
    net.train()
    losses = []
    for data in tqdm.tqdm(loader):
        # xははじめから最後の手前の文字まで
        x = data[:, :-1]
        # yは2文字目から最後の文字まで
        y = data[:, 1:]

        x = x.to("cuda:0")
        y = y.to("cuda:0")

        y_pred, _ = net(x)
        # batchとstepの軸を統合してからCrossEntropyLossに渡す
        loss = loss_f(y_pred.view(-1, vocab_size), y.view(-1))
        net.zero_grad()
        loss.backward()
        opt.step()
        losses.append(loss.item())
    # 現在の損失関数と生成される文章の例を表示
    print(epoch, mean(losses))
    with torch.no_grad():
        print(generate_seq(net, device="cuda:0"))

  0%|          | 0/175 [00:00<?, ?it/s]

0 3.496890231541225
The King said arurcia,NleibnS sd;es Iow ho rs oadhnia r   gCRcngeioshhHwwltcth T  ryan
aa
c
a 
 l
mOC,awUnawi ,eTieln Wtrpieetgonm osmren elasmnsebioa  tem  tletr.g,Kef
rrt ss eh 
o,sras T .opucsbciihr u_ljtcowK usv


  0%|          | 0/175 [00:00<?, ?it/s]

1 3.104111685071673
The King said yoa
RMinr ol,nt to teekrr snidn

:I
r:oe
kis mhit yte s.as mes ttanoa we!erer ier
tac Teo ti o.nenlarkprh esfn Ilal nnad?rohlCbet,

Yasy teannv
cad;u tis+ehsdeceu yas byrit beorm huF't glhp fi:' rinn w


  0%|          | 0/175 [00:00<?, ?it/s]

...

47 1.685372063773019
The King said ficys or here,
Jcord, Engmitform lenses intantas.

PAULINA:
That heard aight seems:
Walk these trud, me? the pldear good the you,
That? Ker and my shall judened to budious?
The same, to scaption but an


  0%|          | 0/175 [00:00<?, ?it/s]

48 1.6821535628182547
The King said spoe all turn yous
when too weir made rocenhardent his trees:
Which we lide you, and with west ad Came yet,
Then fliess him defort Duerys you say my drotob him!
Be got my his mefelfle, ackly forcallife


100%|██████████| 175/175 [00:14<00:00, 12.52it/s]


49 1.6789666305269513
The King said unly a shulfong;
Wo leech o' me daiand. Platan't:'
He bellow pseat, and conforted, now isuth take
And I for the plaw thy say'n theeshe there by natellence
That he calls nom word, we of here at it it da
